### REAL TIME FACE DETECTION AND FACE RECOGNITION

**Objective:** Develop a facial recognition system using the LFW dataset to verify if a detected face matches a specific individual. The system integrates a "door animation" that opens for recognized individuals and closes otherwise.

**Key Highlights:**

* Dataset Streaming:

Use TensorFlow Datasets (TFDS) to stream the Labeled Faces in the Wild (LFW) dataset without downloading.
Dynamically preprocess images (resize and normalize).

* Face Verification:

Create positive and negative pairs for training using the LFW dataset.
Train a Siamese Network to measure similarity between faces.

* Door Animation:

Use OpenCV to simulate a real-world response with an "Open Door" or "Close Door" animation based on recognition results.

* Dynamic and Scalable:

Efficient memory usage by streaming data in batches.
Suitable for real-time face verification applications.

    This exercise combines machine learning, computer vision, and real-world applications in an engaging and interactive way. Perfect for beginners exploring face verification or for showcasing a practical AI project!

In [1]:
## Installing Required Libraries
!pip install tensorflow tensorflow-datasets


### Stream The LFW Dataset

In [2]:
!pip install -U tensorflow-datasets


In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

# Define a function to process the dataset
def preprocess_image(features):
    image = tf.image.convert_image_dtype(features['image'], tf.float32)  # Normalize to [0, 1]
    label = features['label']
    return image, label

# Load the dataset directly from GCS
train_dataset = tfds.load("lfw", split="train", as_supervised=False, shuffle_files=True)

# Apply preprocessing
train_dataset = train_dataset.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)

# Display dataset info
dataset_info = tfds.builder("lfw").info
print(dataset_info)

# Function to visualize images
def visualize_images(dataset, num_images=5):
    plt.figure(figsize=(15, 5))
    image_count = 0

    for batch in dataset:  # Each batch contains (images, labels)
        images, labels = batch  # Unpack images and labels
        for i in range(images.shape[0]):  # Iterate over the batch
            if image_count >= num_images:
                break

            ax = plt.subplot(1, num_images, image_count + 1)
            plt.imshow(tf.keras.utils.array_to_img(images[i]))  # Convert single image to PIL format
            plt.title(labels[i].numpy().decode())  # Decode label
            plt.axis("off")
            image_count += 1

        if image_count >= num_images:
            break

    plt.show()

# Visualize images
visualize_images(train_dataset, num_images=5)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

# Define a function to preprocess the dataset
def preprocess_for_inception(features):
    image = tf.image.resize(features['image'], (299, 299))  # Resize to 299x299
    image = tf.keras.applications.inception_v3.preprocess_input(image)  # Normalize to [-1, 1]
    label = features['label']
    return image, label

# Load the dataset
dataset_builder = tfds.builder("lfw")
dataset_builder.download_and_prepare()

# Create a train dataset
train_dataset = dataset_builder.as_dataset(split='train', as_supervised=False)

# Apply preprocessing for InceptionV3
train_dataset = (
    train_dataset
    .map(preprocess_for_inception, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(32)
    .prefetch(tf.data.AUTOTUNE)
)

# Display dataset info
print(dataset_builder.info)

# Function to visualize preprocessed images
def visualize_images(dataset, num_images=5):
    plt.figure(figsize=(15, 5))
    image_count = 0

    for batch in dataset:  # Each batch contains (images, labels)
        images, labels = batch
        for i in range(images.shape[0]):  # Iterate over the batch
            if image_count >= num_images:
                break

            ax = plt.subplot(1, num_images, image_count + 1)
            # Convert preprocessed image back to [0, 1] for visualization
            display_image = (images[i] + 1.0) / 2.0  # Rescale [-1, 1] to [0, 1]
            plt.imshow(tf.keras.utils.array_to_img(display_image))
            plt.title(labels[i].numpy().decode("utf-8"))
            plt.axis("off")
            image_count += 1

        if image_count >= num_images:
            break

    plt.show()

# Visualize preprocessed images
visualize_images(train_dataset, num_images=5)


### Cropping Faces and Preparing Data for Transfer Learning

In [ ]:
import tensorflow_datasets as tfds
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Load the Haar Cascade
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the LFW dataset
lfw_dataset = tfds.load('lfw', split='train', as_supervised=False)

# Function to process and detect faces
def detect_faces_from_lfw(sample):
    image = sample['image'].numpy()  # Get the image as a NumPy array
    label = sample['label'].numpy().decode()  # Decode the label

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Detect faces
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw rectangles around detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)

    # Display the image with detections
    plt.imshow(image)
    plt.title(f"Label: {label}")
    plt.axis("off")
    plt.show()

# Test the function with the first few images
for i, sample in enumerate(lfw_dataset.take(5)):
    detect_faces_from_lfw(sample)


In [ ]:
cropped_faces = []  # To store cropped face images
labels = []  # To store corresponding labels

# Function to crop and save faces
def crop_and_store_faces(sample):
    image = sample['image'].numpy()
    label = sample['label'].numpy().decode()

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Detect faces
    faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Crop faces
    for (x, y, w, h) in faces:
        cropped_face = image[y:y+h, x:x+w]
        cropped_faces.append(cropped_face)
        labels.append(label)  # Store the label for this face

# Process all images in the dataset
for sample in lfw_dataset.take(500):  # Limiting to 500 samples for demonstration
    crop_and_store_faces(sample)

# Resize cropped faces to the input size required by InceptionV3 (299x299)
cropped_faces_resized = [
    cv2.resize(face, (299, 299)) for face in cropped_faces
]

# Normalize to [0, 1]
cropped_faces_normalized = np.array(cropped_faces_resized) / 255.0

# Encode labels into integers
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(
    cropped_faces_normalized, encoded_labels, test_size=0.2, random_state=42
)

print(f"Training data: {x_train.shape}, Validation data: {x_val.shape}")
print(f"Number of classes: {len(label_encoder.classes_)}")


## Code: Training the Recognition Model


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Load InceptionV3 base model
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the base model layers
base_model.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(len(label_encoder.classes_), activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display model summary
model.summary()

# Train the model
history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=10,
    batch_size=32
)


### Face Detection Using OpenCV
1. Load the Haar Cascade


In [ ]:
# Unfreeze some layers of the base model
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Recompile with a lower learning rate
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model
fine_tune_history = model.fit(
    x_train, y_train,
    validation_data=(x_val, y_val),
    epochs=5,
    batch_size=32
)


In [ ]:
# Save the model
model.save('face_recognition_model.keras')

# Evaluate on the validation set
val_loss, val_acc = model.evaluate(x_val, y_val)
print(f"Validation Accuracy: {val_acc:.2f}")


In [ ]:
import cv2
import tensorflow as tf
import numpy as np

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load trained model
model = tf.keras.models.load_model('face_recognition_model.keras')

# Class labels
class_names = ['Barack Obama', 'George Bush', 'Other']


In [ ]:
pip install streamlit

In [ ]:
from PIL import Image, ImageDraw
import streamlit as st

def draw_door(opened):
    img = Image.new("RGB", (500, 300), "white")
    draw = ImageDraw.Draw(img)

    # Door frame
    draw.rectangle([50, 50, 450, 250], outline="black", width=5)

    if opened:
        # Door open
        draw.rectangle([300, 50, 450, 250], fill="white")
        text = "Door Open"
    else:
        # Door closed
        draw.rectangle([50, 50, 450, 250], fill="brown")
        text = "Door Closed"

    draw.text((200, 270), text, fill="black")
    return img


In [ ]:
# Define the app code as a string
app_code = '''
import streamlit as st
from PIL import Image
import cv2
import numpy as np
import tensorflow as tf

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load trained model
model = tf.keras.models.load_model('face_recognition_model.keras')

# Class labels
class_names = ['Barack Obama', 'George Bush', 'Other']

# Door animation
def draw_door(opened):
    from PIL import Image, ImageDraw
    img = Image.new("RGB", (500, 300), "white")
    draw = ImageDraw.Draw(img)

    # Door frame
    draw.rectangle([50, 50, 450, 250], outline="black", width=5)

    if opened:
        # Door open
        draw.rectangle([300, 50, 450, 250], fill="white")
        text = "Door Open"
    else:
        # Door closed
        draw.rectangle([50, 50, 450, 250], fill="brown")
        text = "Door Closed"

    draw.text((200, 270), text, fill="black")
    return img

st.title("Face Recognition Door System")

# Upload an image
uploaded_file = st.file_uploader("Upload an image", type=["jpg", "png", "jpeg"])

if uploaded_file:
    # Read image
    image = np.array(Image.open(uploaded_file))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        # Crop and preprocess face
        face = image[y:y + h, x:x + w]
        face = cv2.resize(face, (299, 299))  # Inception V3 input size
        face = np.expand_dims(face, axis=0) / 255.0

        # Predict
        predictions = model.predict(face)
        predicted_label = class_names[np.argmax(predictions)]

        # Draw face box
        cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(image, predicted_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Door animation
    if predicted_label in ["Barack Obama", "George Bush"]:
        st.image(draw_door(opened=True))
    else:
        st.image(draw_door(opened=False))

    # Display image with annotations
    st.image(image, caption="Processed Image", use_column_width=True)
'''

# Save the app code to a file
with open("app.py", "w") as file:
    file.write(app_code)


In [ ]:
# List of required libraries
requirements = """
streamlit
tensorflow
opencv-python
pillow
numpy
"""

# Write to requirements.txt
with open("requirements.txt", "w") as file:
    file.write(requirements)

print("requirements.txt has been created.")


In [ ]:
# Install Academic Torrents
!pip install academictorrents

import academictorrents as at
import os
import zipfile

# Directory to store the downloaded dataset
dataset_dir = "./yale_faces"

# Ensure the directory exists
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

# Torrent hash for the Extended Yale Face Database B
torrent_hash = "06e479f338b56fa5948c40287b66f68236a14612"

# Download the dataset
print("Downloading the dataset...")
dataset_path = at.get(torrent_hash, dataset_dir)
print(f"Dataset downloaded to: {dataset_path}")

# Extract the dataset
if not os.path.exists("./CroppedYale"):
    print("Extracting dataset...")
    with zipfile.ZipFile(dataset_path, "r") as zip_ref:
        zip_ref.extractall("./")
    print("Extraction complete!")

# Confirm extracted content
print("Content extracted. Listing files...")
!ls -lh ./CroppedYale


In [ ]:
import os
import requests
import zipfile

# Directory to save the dataset
dataset_dir = "/content/WIDER_FACE"
os.makedirs(dataset_dir, exist_ok=True)

def download_file(url, output_path, expected_size=None):
    """Download a file and verify its size."""
    if os.path.exists(output_path):
        print(f"File already exists: {output_path}")
        return

    print(f"Downloading {url}...")
    with requests.get(url, stream=True) as response:
        response.raise_for_status()  # Raise an error for failed downloads
        with open(output_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                f.write(chunk)
    print(f"Download complete: {output_path}")

    # Check file size if expected_size is provided
    if expected_size and os.path.getsize(output_path) != expected_size:
        raise ValueError(f"Downloaded file size does not match the expected size: {expected_size} bytes")

def download_and_unzip(url, output_dir, expected_size=None):
    """Download and unzip a file from the given URL."""
    filename = os.path.join(output_dir, url.split("/")[-1])
    extracted_dir = filename.replace(".zip", "")

    # Check if the file is already extracted
    if os.path.exists(extracted_dir):
        print(f"Data already extracted: {extracted_dir}")
        return

    # Download file
    download_file(url, filename, expected_size)

    # Unzip the file
    print(f"Extracting {filename}...")
    try:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(output_dir)
        print(f"Extraction complete: {extracted_dir}")
    except zipfile.BadZipFile:
        print(f"Error: {filename} is not a valid zip file. Please redownload.")
        raise

# URLs for the WIDER FACE dataset
wider_face_train_url = "http://shuoyang1213.me/WIDERFACE/WIDER_train.zip"
wider_face_val_url = "http://shuoyang1213.me/WIDERFACE/WIDER_val.zip"
wider_face_anno_url = "http://shuoyang1213.me/WIDERFACE/wider_face_split.zip"

# Expected file sizes in bytes
expected_sizes = {
    wider_face_train_url: 11000000000,  # ~11 GB
    wider_face_val_url: 1400000000,    # ~1.4 GB
    wider_face_anno_url: 20000000      # ~20 MB
}

# Download and unzip each dataset
for url, expected_size in expected_sizes.items():
    download_and_unzip(url, dataset_dir, expected_size)

print("All files downloaded and extracted successfully!")


In [ ]:

import zipfile
import os

# Path to the ZIP file
zip_path = "/content/WIDER_FACE/WIDER_val.zip"

# Directory to extract the contents
extract_dir = "/content/WIDER_FACE/WIDER_val"

# Check if the file is a valid ZIP
if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # Extract all contents
        zip_ref.extractall(extract_dir)
        print(f"Extracted files to {extract_dir}")
else:
    print(f"The file at {zip_path} is not a valid ZIP file.")




